### Helper Classes

First we get all of our helper modules. The prepare_EMG module will prepare the EMG data for phoneme recognition. The prepare_outputs module will prepare our target labels and align them with our EMG data. The module 'prepare_data' will help us read data from CSV into a dataframe. Finally, 'vis' will help visualize EMG data in both time and frequency domains. 

In [1]:
%load_ext autoreload
%autoreload 2

import prepare_EMG, prepare_outputs, prepare_data, vis
# autodetector = Output_Prep.detector
EMG_Prep = prepare_EMG.EMG_preparer()
# Output_Prep = prepare_outputs.output_preparer(subvocal_detector = autodetector, window_size=30.0)
Output_Prep = prepare_outputs.output_preparer()

Data_Prep = prepare_data.data_preparer()



In [2]:
singles = Data_Prep.load_singletons(1)
print(singles.keys())

File b'simple-svr-data/raspy-1' does not exist
dict_keys(['dusty', 'march', 'direful', 'complete', 'superb', 'poised', 'wait', 'quaint', 'save', 'copy', 'interest', 'separate', 'bright', 'utter', 'bored', 'nondescript', 'license', 'vest', 'dance', 'money', 'languid', 'swim', 'enthusiastic', 'quartz', 'planes', 'spiritual', 'imperfect', 'coal', 'hobbies', 'sound', 'bow', 'squirrel', 'push', 'treatment', 'mine', 'precede', 'weather', 'amazing', 'round', 'stingy', 'signal', 'marry', 'country', 'uncle', 'dust', 'certain', 'loose', 'knock', 'advice', 'confuse', 'animated', 'loving', 'feeling', 'absorbing', 'trick', 'spare', 'rod', 'caption', 'throne', 'clumsy', 'vague', 'tow', 'hang', 'rely', 'tired', 'barbarous', 'pan', 'innocent', 'combative', 'low', 'rub', 'mixed', 'actually', 'faulty', 'thirsty', 'dam', 'doubtful', 'flowers', 'defective', 'frogs', 'outstanding', 'ducks', 'icicle', 'fry', 'load', 'cracker', 'efficient', 'hop', 'fax', 'fancy', 'reading', 'real', 'addicted', 'motion', 'cle

### Labeling the Data

First, we generate the phoneme and articulatory feature labels from each word. We'll use these to process the data in each file based on the length of the file and how many phonemes it should contain. We scale the FFT windows relative to the length of time we expect an even distribution of phonemes across the file to have. 

In [3]:
# data_1 = Data_Prep.load('Sat Mar  4 00:44:23 2017')
# data_2 = Data_Prep.load('Sat Mar  4 00:45:02 2017')
# data_3 = Data_Prep.load('Sat Mar  4 00:45:47 2017')
# data_4 = Data_Prep.load('Sat Mar  4 00:47:01 2017')
# data_5 = Data_Prep.load('Sat Mar  4 00:47:36 2017')
# data_6 = Data_Prep.load('Sat Mar  4 00:48:09 2017')
# data_7 = Data_Prep.load('Sat Mar  4 00:49:05 2017')
# data_8 = Data_Prep.load('Sat Mar  4 00:49:41 2017')
# data_9 = Data_Prep.load('Sat Mar  4 00:50:22 2017')
# data_10 = Data_Prep.load('Sat Mar  4 00:51:17 2017')
# data_11 = Data_Prep.load('Sat Mar  4 00:52:02 2017')
# data_12 = Data_Prep.load('Sat Mar  4 00:52:38 2017')
# data_13 = Data_Prep.load('Sat Mar  4 00:53:24 2017')
# data_14 = Data_Prep.load('Sat Mar  4 00:53:51 2017')
# data_15 = Data_Prep.load('Sat Mar  4 00:54:25 2017')
# data_16 = Data_Prep.load('Sat Mar  4 00:54:57 2017')
# data_17 = Data_Prep.load('Sat Mar  4 00:56:01 2017')
# data_18 = Data_Prep.load('Sat Mar  4 00:56:35 2017')
# data_19 = Data_Prep.load('Sat Mar  4 00:57:21 2017')
# data_20 = Data_Prep.load('Sat Mar  4 00:57:49 2017')
# data_21 = Data_Prep.load('Sat Mar  4 00:58:59 2017')
# data_22 = Data_Prep.load('Sat Mar  4 00:59:53 2017')

# data_list = [data_1, data_2, data_3, data_4, data_5, data_6, data_7, data_8, data_9, data_10, data_11, data_12, data_13, data_14, data_15, data_16, data_17, data_18, data_19, data_20, data_21, data_22]
labels = {}
windows = {}
for word in singles:
    label = Output_Prep.transform(word)
    num_phonemes = label.shape[0]
    labels[word] = label
    windows[word] = EMG_Prep.process(singles[word],num_phonemes)




In [4]:
test_word = 'license'
test_word2 = 'money'
test_EMG = windows[test_word]
test_EMG2 = windows[test_word2]

print(test_EMG.head(6), labels[test_word], test_EMG2.head(), labels[test_word2])

       0.0       10.0      10.0      20.0      20.0      30.0      30.0   \
0  43.492969  0.020570  2.259229  1.954676  1.681519  1.445389  2.413685   
1  47.746875  2.017764  0.095421  0.633403  0.322932  2.490014  1.047570   
2  43.325391  1.125540  0.838279  0.147355  0.309692  0.321898  0.573655   
3  42.680859  1.142250  0.402145  1.505252  0.509298  0.329306  0.888195   
4  44.421094  0.062747  1.107197  0.216720  0.385406  1.835386  1.378310   
5  44.098828  0.190692  0.318395  1.432442  0.888541  0.892307  0.651676   

      40.0      40.0      50.0     ...        450.0     460.0     460.0  \
0  0.423931  1.066141  1.716362    ...     0.394087  0.043894  0.195630   
1  0.822770  4.314441  0.909039    ...     0.924509  0.736296  1.002601   
2  4.172037  2.061649  0.181364    ...     1.295144  0.545687  1.338983   
3  1.115764  2.908927  1.167312    ...     0.145217  0.530457  0.348105   
4  1.094253  2.286328  0.503626    ...     0.123156  2.375713  1.121218   
5  2.017346  2.06

In [31]:
import pandas
%autoreload 2

# num_files = len(data_list)
# labels_frame = pandas.read_csv('austen_subvocal.csv')
# trans_labels = Output_Prep.transform(labels_frame.iloc[0][0])
# data_1_proc = EMG_Prep.process(data_1)
# aligned_data, trans_labels= Output_Prep.zip(data_1_proc, trans_labels, repeat=3)

# for file in range(1, num_files):
#     trans_labels_iter = Output_Prep.transform(labels_frame.iloc[file][0])
#     data_proc_iter = EMG_Prep.process(data_list[file])
#     aligned_data_iter, trans_labels_iter = Output_Prep.zip(data_proc_iter, trans_labels_iter, repeat=3)

#     aligned_data = aligned_data.append(aligned_data_iter)
#     trans_labels = trans_labels.append(trans_labels_iter)
    
# print('Aligned Data shape:',aligned_data.shape,'Trans labels shape:',trans_labels.shape)
y = pandas.DataFrame()
X = pandas.DataFrame()
for word in labels:
#     append labels to the master label dataframe
    label_frame = labels[word]
    y = y.append(label_frame)
#     Use phonemes to name each series in 'windows' for that word
    window_frame = windows[word]
    if len(label_frame.axes[0]):
        window_frame = window_frame.rename_axis(lambda x: label_frame.axes[0][x])
        X = X.append(window_frame)
    else:
        print('no labels for:',word)

#     for row in range(label_frame.shape[0]):
        
#         new_row = pandas.Series(window_frame.iloc[row])
#         new_row.name = label_frame.iloc[row].name
#         X = X.append(new_row)
#     Append windows to a master window dataframe
# X = X.rename_axis(list(zip(X.axes[0],y.axes[0])))

no labels for: direful
no labels for: wrathful


In [32]:
# print(X.head(18), y.head(18))
from sklearn.preprocessing import scale,normalize
from sklearn.decomposition import PCA

pca = PCA(n_components=10, random_state=9)
X_reduced = pca.fit_transform(X)
X_scaled = scale(X_reduced)
X_normalized = normalize(X_scaled)

X_normalized = pandas.DataFrame(X_normalized)
X_normalized = X_normalized.rename_axis(lambda x: 'pc-'+str(x), axis='columns')
print(X_normalized)


         pc-0      pc-1      pc-2      pc-3      pc-4      pc-5      pc-6  \
0   -0.266407 -0.207434 -0.370914  0.220957  0.075752  0.459644 -0.403992   
1   -0.125598 -0.237161 -0.046843  0.470252 -0.316733  0.249924 -0.135016   
2   -0.151326 -0.332380 -0.203439  0.837413 -0.027055 -0.141695  0.059433   
3   -0.142370  0.170402 -0.517729  0.439290 -0.161981 -0.462959  0.077455   
4   -0.261900 -0.161843  0.397244 -0.185472 -0.526234 -0.177745  0.081501   
5   -0.186009  0.220340 -0.593467  0.150454  0.001783  0.076995  0.079525   
6   -0.153890 -0.172706 -0.301570  0.756725 -0.019435 -0.264306 -0.035329   
7   -0.161444  0.197694 -0.371258  0.515841 -0.196536 -0.626474  0.099488   
8   -0.250105  0.576111 -0.054499  0.140373 -0.520847 -0.359754 -0.144183   
9   -0.097954 -0.216014 -0.431797 -0.083493 -0.205396  0.261457  0.281590   
10  -0.243750  0.751185 -0.073861 -0.338155  0.003141  0.323720  0.188731   
11  -0.176715  0.103583  0.402070  0.285961  0.549390 -0.368358 -0.390115   

### AF Extractor Models

These models will be optimized for extracting AF's from the data, before passing those AF's onto an MLPC for identifying the most likely phoneme. 

In [33]:
# Prepare lists of parameters for our GridSearch
# First, our layer sizes
layer_sizes = []
for i in range(2,5):
    for j in range(0,180,30):
        if j:
            tup = []
            for k in range(i):
                tup.append(j)
            layer_sizes.append(tuple(tup))
print('number layer sizes:',len(layer_sizes),'here be layer sizes',layer_sizes)

# Next, our alpha values
alphas = [0.0001,1,1000]

number layer sizes: 15 here be layer sizes [(30, 30), (60, 60), (90, 90), (120, 120), (150, 150), (30, 30, 30), (60, 60, 60), (90, 90, 90), (120, 120, 120), (150, 150, 150), (30, 30, 30, 30), (60, 60, 60, 60), (90, 90, 90, 90), (120, 120, 120, 120), (150, 150, 150, 150)]


### Preparing GridSearch and Assesing Stock MLPC as AF extractor models

We setup the objects for performing gridsearch on each one of the Articulatory Feature Extractor models. We also train untuned, stock MLPC models to serve as a performance baseline. We will compare the performance of these baseline, untuned models to our gridsearched models to determine whether gridsearch has in fact improved the model parameters for each AF extractor. 

In [34]:
from sklearn.neural_network import MLPClassifier as MLPC
# Import other models to try for feature extraction
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

import copy

X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.15, random_state=12)

combined_features = FeatureUnion([
    ('pca',PCA(random_state=18)),
    ('kbest',SelectKBest(k=1))
])

pipeline = Pipeline([
    ('features', combined_features),
    ('model', MLPC(random_state=12))
])


param_grid = {
#     'features__pca__n_components':[10,20,50],
    'model__solver':['adam'],
    'model__hidden_layer_sizes':layer_sizes,
    'model__activation':['relu'],
    'model__alpha': alphas,
    'model__max_iter':[200]
}


grid_search = GridSearchCV(pipeline, param_grid, n_jobs=-1)

manner_classifier = MLPC(solver='adam',random_state=3)
manner_classifier.fit(X_train, y_train['manner'])
m_score = manner_classifier.score(X_test, y_test['manner'])

place_classifier = MLPC(solver='adam',random_state=6)
place_classifier.fit(X_train, y_train['place'])
p_score = place_classifier.score(X_test, y_test['place'])

height_classifier = MLPC(solver='adam',random_state=9)
height_classifier.fit(X_train, y_train['height'])
h_score = height_classifier.score(X_test, y_test['height'])

vowel_classifier = MLPC(solver='adam',random_state=12)
vowel_classifier.fit(X_train, y_train['vowel'])
v_score = vowel_classifier.score(X_test, y_test['vowel'])

print('manner score:',m_score,'place score:',p_score,'height score:',h_score,'vowel score:',v_score)
# print(data_1_proc.head(50), trans_labels['manner'].head(50))

/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


manner score: 0.357142857143 place score: 0.258928571429 height score: 0.5 vowel score: 0.633928571429


In [9]:
manner_classifier2 = copy.deepcopy(grid_search)
manner_classifier2.fit(X_train, y_train['manner'])
m_score2 = manner_classifier2.score(X_test, y_test['manner'])

print('manner score:',m_score2)

/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/py

manner score: 0.383928571429


In [10]:
place_classifier2 = copy.deepcopy(grid_search)
place_classifier2.fit(X_train, y_train['place'])
p_score2 = place_classifier2.score(X_test, y_test['place'])

print('place score:',p_score2)

/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/py

place score: 0.276785714286


In [11]:
height_classifier2 = copy.deepcopy(grid_search)
height_classifier2.fit(X_train, y_train['height'])
h_score2 = height_classifier2.score(X_test, y_test['height'])

print('height score:',h_score2)

/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/py

height score: 0.473214285714


In [12]:
vowel_classifier2 = copy.deepcopy(grid_search)
vowel_classifier2.fit(X_train, y_train['vowel'])
v_score2 = vowel_classifier2.score(X_test, y_test['vowel'])

print('vowel score:',v_score2)

/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/home/brian/anaconda3/lib/py

vowel score: 0.616071428571


In [75]:
# Experiment with PCA here

from sklearn.decomposition import PCA


manner_union = FeatureUnion([('pca',PCA(n_components=0.03)),('kbest',SelectKBest(k=1))])
manner_reduced_data = manner_union.fit_transform(X, y['manner'])
manner_X_train, manner_X_test, manner_y_train, manner_y_test = train_test_split(manner_reduced_data, y, test_size=0.15, random_state=12)


manner_classifier3 = MLPC(solver='adam',alpha=0.000001,hidden_layer_sizes=(90,90,90,90),random_state=3,max_iter=300)
manner_classifier3.fit(manner_X_train, manner_y_train['manner'])
m_score3 = manner_classifier3.score(manner_X_test, manner_y_test['manner'])
print('manner score:',m_score3)

manner score: 0.392857142857


In [93]:
place_union = FeatureUnion([('pca',PCA(n_components=0.9)),('kbest',SelectKBest(k=1))])
place_reduced_data = place_union.fit_transform(X, y['place'])
place_X_train, place_X_test, place_y_train, place_y_test = train_test_split(place_reduced_data, y, test_size=0.15, random_state=12)


place_classifier3 = MLPC(solver='adam',alpha=100,hidden_layer_sizes=(100,100,100),random_state=6,max_iter=900)
place_classifier3.fit(place_X_train, place_y_train['place'])
p_score3 = place_classifier3.score(place_X_test, place_y_test['place'])
print('place classifier score:',p_score3)

place classifier score: 0.276785714286


In [101]:
height_union = FeatureUnion([('pca',PCA(n_components=0.5)),('kbest',SelectKBest(k=1))])
height_reduced_data = height_union.fit_transform(X, y['height'])
height_X_train, height_X_test, height_y_train, height_y_test = train_test_split(height_reduced_data, y, test_size=0.15, random_state=12)


height_classifier3 = MLPC(solver='adam',alpha=1,hidden_layer_sizes=(1),random_state=12,max_iter=300)
height_classifier3.fit(height_X_train, height_y_train['height'])
h_score3 = height_classifier3.score(height_X_test, height_y_test['height'])
print('height score:',h_score3)

height score: 0.473214285714


/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [106]:
vowel_union = FeatureUnion([('pca',PCA(n_components=0.3)),('kbest',SelectKBest(k=1))])
vowel_reduced_data = vowel_union.fit_transform(X, y['vowel'])
vowel_X_train, vowel_X_test, vowel_y_train, vowel_y_test = train_test_split(vowel_reduced_data, y, test_size=0.15, random_state=12)


vowel_classifier3 = MLPC(solver='adam',alpha=100,hidden_layer_sizes=(90,90),random_state=12,max_iter=600)
vowel_classifier3.fit(vowel_X_train, vowel_y_train['vowel'])
v_score3 = vowel_classifier3.score(vowel_X_test, vowel_y_test['vowel'])
print('vowel score:',v_score3)

vowel score: 0.616071428571


In [122]:
pho_score = phoneme_classifier.score(X, phoneme_labels)
print(pho_score)

ValueError: shapes (745,100) and (130,90) not aligned: 100 (dim 1) != 130 (dim 0)

In [35]:
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.preprocessing import MultiLabelBinarizer as MLB
from sklearn.preprocessing import OneHotEncoder as OHE
from collections import Counter

m_count = Counter()
p_count = Counter()
h_count = Counter()
v_count = Counter()

for row in range(y.shape[0]):
    m_count.update([y.iloc[row]['manner']])
    p_count.update([y.iloc[row]['place']])
    h_count.update([y.iloc[row]['height']])
    v_count.update([y.iloc[row]['vowel']])
    
counters = [m_count,p_count,h_count,v_count]

feature_dict = {}
for count in counters:
    current = 0
    for feature in count.keys():
        feature_dict[feature] = current
        current += 1
        
print(feature_dict)

{'voiced-stop': 0, 'vowel': 1, 'fricative': 2, 'stop': 3, 'nasal': 4, 'approximant': 5, 'voiced-fricative': 6, 'aspirated': 7, 'alveolar': 0, 'mid': 1, 'front': 2, 'labial': 3, 'back': 4, 'retroflex': 5, 'dorsal': 6, 'lateral': 7, 'mid-back': 8, 'mid-front': 9, 'dental': 10, 'uknown': 11, 'max': 0, 'very high': 2, 'low': 3, 'mid-low': 4, 'high': 5, 'very-high': 6, 'mid-high': 7, 'no': 0, 'yes': 1}


In [36]:
num_labels = copy.deepcopy(y)
for row in range(y.shape[0]):
    m_feat = y.iloc[row]['manner']
    p_feat = y.iloc[row]['place']
    h_feat = y.iloc[row]['height']
    v_feat = y.iloc[row]['vowel']
    num_labels.iloc[row]['manner'] = feature_dict[m_feat]
    num_labels.iloc[row]['place'] = feature_dict[p_feat]
    num_labels.iloc[row]['height'] = feature_dict[h_feat]
    num_labels.iloc[row]['vowel'] = feature_dict[v_feat]
num_labels.head()

,manner,place,height,vowel
D,0,0,0,0
AH,1,1,1,1
S,2,0,0,0
T,3,0,0,0
IY,1,2,2,1


In [37]:
encoder = OHE()
new_labels = encoder.fit_transform(num_labels)
print(new_labels.toarray())
enc_labels = pandas.DataFrame(new_labels.toarray())
print(enc_labels.head())

[[ 1.  0.  0. ...,  0.  1.  0.]
 [ 0.  1.  0. ...,  0.  0.  1.]
 [ 0.  0.  1. ...,  0.  1.  0.]
 ..., 
 [ 0.  1.  0. ...,  0.  0.  1.]
 [ 0.  1.  0. ...,  0.  0.  1.]
 [ 0.  0.  1. ...,  0.  1.  0.]]
    0    1    2    3    4    5    6    7    8    9  ...    20   21   22   23  \
0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0 ...   1.0  0.0  0.0  0.0   
1  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0 ...   0.0  1.0  0.0  0.0   
2  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0 ...   1.0  0.0  0.0  0.0   
3  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  1.0  0.0 ...   1.0  0.0  0.0  0.0   
4  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 ...   0.0  0.0  1.0  0.0   

    24   25   26   27   28   29  
0  0.0  0.0  0.0  0.0  1.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  1.0  
2  0.0  0.0  0.0  0.0  1.0  0.0  
3  0.0  0.0  0.0  0.0  1.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  1.0  

[5 rows x 30 columns]


In [38]:
cols = list(X_normalized.axes[1]) + list(enc_labels.axes[1])
# print(cols)

phoneme_inputs = pandas.DataFrame(columns=cols)
phoneme_labels = y.axes[0]

for row in range(X.shape[0]):
    new_row = pandas.Series(X_normalized.iloc[row]).append(enc_labels.iloc[row])
    new_row.name = X.iloc[row].name
#     print(new_row.axes)
#     new_row = new_row.reindex(range(len(new_row)))
#     print(new_row)
#     new_row = new_row.rename(row)

    phoneme_inputs = phoneme_inputs.append(new_row)

pho_X_train, pho_X_test, pho_y_train, pho_y_test = train_test_split(phoneme_inputs,phoneme_labels, test_size=0.15, random_state=12)

phoneme_classifier = MLPC(solver='adam',alpha=0.1,hidden_layer_sizes=(15),random_state=6, max_iter=400)
phoneme_classifier.fit(pho_X_train, pho_y_train)

MLPClassifier(activation='relu', alpha=0.1, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=15, learning_rate='constant',
       learning_rate_init=0.001, max_iter=400, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=6, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [39]:
pho_score = phoneme_classifier.score(pho_X_test, pho_y_test)
print(pho_score)


0.910714285714


In [42]:
# feature_extractors = FeatureUnion([
#     ('manner_extractor',manner_classifier2.best_estimator_),
#     ('place_extractor',place_classifier2.best_estimator_),
#     ('height_extractor',height_classifier2.best_estimator_),
#     ('vowel_extractor',vowel_classifier2.best_estimator_),
#     ('pho_pca',PCA())
# ])

# pho_pipe = Pipeline([
#     ('AF_extractors',feature_extractors),
#     ('pho_classifier',MLPC())
# ])
manner_inputs = manner_classifier.predict(pho_X_test.iloc[:,0:10])
place_inputs = place_classifier.predict(pho_X_test.iloc[:,0:10])
height_inputs = height_classifier.predict(pho_X_test.iloc[:,0:10])
vowel_inputs = vowel_classifier.predict(pho_X_test.iloc[:,0:10])
raw_inputs = copy.deepcopy(y)
for row in range(len(manner_inputs)):

    raw_inputs.iloc[row]['manner'] = manner_inputs[row]
    raw_inputs.iloc[row]['place'] = place_inputs[row]
    raw_inputs.iloc[row]['height'] = height_inputs[row]
    raw_inputs.iloc[row]['vowel'] = vowel_inputs[row]
    
# print(raw_inputs.head(300))

test_num_labels = copy.deepcopy(raw_inputs)
for row in range(pho_X_test.shape[0]):
    m_feat = raw_inputs.iloc[row]['manner']
#     print(m_feat)
    p_feat = raw_inputs.iloc[row]['place']
    h_feat = raw_inputs.iloc[row]['height']
    v_feat = raw_inputs.iloc[row]['vowel']
    test_num_labels.iloc[row]['manner'] = feature_dict[m_feat]
    test_num_labels.iloc[row]['place'] = feature_dict[p_feat]
    test_num_labels.iloc[row]['height'] = feature_dict[h_feat]
    test_num_labels.iloc[row]['vowel'] = feature_dict[v_feat]
print(test_num_labels.head(100))

encoder = OHE()
test_new_labels = encoder.fit_transform(test_num_labels)
# print(new_labels.toarray())
test_enc_labels = pandas.DataFrame(test_new_labels.toarray())
# print(enc_labels.head())

cols = list(X.axes[1]) + list(test_enc_labels.axes[1])
# print(cols)

test_pho_inputs = pandas.DataFrame()
phoneme_labels = y.axes[0]

for row in range(pho_X_test.shape[0]):
    new_row = pho_X_test.iloc[row,0:10].append(test_enc_labels.iloc[row])
    new_row.name = pho_X_test.iloc[row].name
#     print(new_row)
    test_pho_inputs = test_pho_inputs.append(new_row)

# pho_pipe.fit(pho_X_train, pho_y_train)
pho_test_score = phoneme_classifier.score(test_pho_inputs, pho_y_test)
print('phoneme model test score:',pho_test_score)

   manner place height vowel
D       1     0      0     0
AH      1     2      0     0
S       1     0      0     0
T       1     0      0     0
IY      1     0      0     0
M       1     1      1     1
AA      1     0      0     0
R       1     0      0     1
CH      1     0      0     0
K       1     2      0     0
AH      1     0      0     0
M       1     0      0     0
P       1     0      0     0
L       1     0      0     0
IY      1     0      0     0
T       1     1      1     1
S       1     0      0     0
UH      1     0      0     0
P       1     1      0     0
ER      3     0      0     0
B       3     0      0     0
P       1     0      0     0
OY      1     0      0     0
Z       1     0      0     0
D       1     0      0     0
W       1     0      0     0
EY      1     0      0     1
T       1     4      0     0
K       1     7      2     1
W       1     0      0     0
..    ...   ...    ...   ...
K       3     0      0     0
R       1     0      0     0
IH      1     

ValueError: could not convert string to float: 'no'

In [22]:
print(vowel_inputs[:300])

['no' 'no' 'no' 'no' 'no' 'yes' 'no' 'yes' 'no' 'no' 'no' 'no' 'no' 'no'
 'no' 'yes' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'yes' 'no'
 'yes' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'yes' 'yes'
 'no' 'no' 'yes' 'no' 'no' 'yes' 'yes' 'yes' 'no' 'no' 'no' 'no' 'no' 'no'
 'no' 'no' 'no' 'yes' 'yes' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no'
 'no' 'no' 'no' 'yes' 'no' 'yes' 'no' 'yes' 'no' 'no' 'no' 'no' 'no' 'no'
 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'no' 'yes' 'no' 'no' 'yes' 'no' 'no'
 'no' 'yes' 'no' 'no' 'no' 'no' 'no' 'yes' 'no' 'no' 'no' 'no' 'no' 'no']


In [23]:
print(pho_X_test.iloc[:,0:100].head(300))

        pc-0      pc-1      pc-2      pc-3      pc-4      pc-5      pc-6  \
P  -0.170578 -0.047012  0.496045 -0.293460 -0.268649 -0.104636  0.315886   
M  -0.176715  0.103583  0.402070  0.285961  0.549390 -0.368358 -0.390115   
N  -0.351283 -0.236495  0.559604 -0.505376 -0.352878  0.079643 -0.123634   
Z  -0.235929 -0.094894  0.586030  0.031012 -0.181180 -0.468536  0.093427   
K   0.708101  0.032783  0.221345 -0.307937 -0.077766  0.069791  0.376905   
ER -0.047511  0.240414  0.493199  0.187694  0.218136  0.377744  0.288087   
L   0.404969 -0.003699  0.161292  0.337634 -0.158560 -0.099604 -0.292701   
T  -0.412289 -0.156800  0.328141  0.075494 -0.145127 -0.258340 -0.048388   
IH  0.716017 -0.065580  0.014837 -0.176467  0.055059  0.162022  0.570452   
N  -0.359159 -0.558895  0.194941  0.132525  0.414076  0.133935 -0.294369   
W  -0.211273  0.425017 -0.334271  0.506617 -0.298250 -0.208356  0.170698   
R  -0.290068  0.460540  0.030968 -0.353409 -0.604025  0.227377 -0.072891   
EH  0.424518

In [24]:
from sklearn.metrics import f1_score

Pho_F1 = f1_score(pho_y_test, phoneme_classifier.predict(test_pho_inputs), average='micro')
print(Pho_F1)

NameError: name 'test_pho_inputs' is not defined

In [25]:
manner_test_predict = manner_classifier.predict(X_test)
print(manner_test_predict, y_test)

['vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel'
 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel'
 'vowel' 'stop' 'stop' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel'
 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'stop' 'stop' 'vowel'
 'vowel' 'stop' 'vowel' 'stop' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel'
 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'stop'
 'fricative' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel'
 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'stop'
 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'stop'
 'vowel' 'vowel' 'stop' 'vowel' 'vowel' 'vowel' 'stop' 'vowel' 'vowel'
 'vowel' 'vowel' 'vowel' 'vowel' 'vowel' 'fricative' 'vowel' 'vowel'
 'vowel' 'vowel' 'vowel' 'stop' 'vowel' 'vowel' 'vowel' 'vowel' 'vowel'
 'vowel' 'stop' 'vowel' 'stop' 'vowel' 'vowel']               manner      place     height vowel
P               stop     labial        max    n